In [47]:
import json
import pandas as pd
import numpy as np
import time
import requests

In [48]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [49]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [50]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result'] 

In [51]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [52]:
def getStakeAndRewards():
    stakes = dict()
    rewards = dict()
    shards = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            address = i['validator']['address']
            reward_accumulated = i['lifetime']['reward-accumulated']
            rewards[address] = reward_accumulated
            by_shard_metrics = i['metrics']['by-bls-key']
            v_stakes = dict()
            v_shards = dict()
            for by_shard_metric in by_shard_metrics:
                bls_key = by_shard_metric['key']['bls-public-key']
                e_stake = float(by_shard_metric['key']['effective-stake'])
                shard_id = by_shard_metric['key']['shard-id']
                v_stakes[bls_key] = e_stake
                v_shards[bls_key] = shard_id
            stakes[address] = v_stakes
            shards[address] = v_shards
    return rewards, stakes, shards

In [53]:
def getRewards():
    rewards = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            address = i['validator']['address']
            reward_accumulated = i['lifetime']['reward-accumulated']
            rewards[address] = reward_accumulated
    return rewards

In [54]:
def proportional(l1, l2):
    return l1 == l2

In [55]:
def extract(lst): 
    return [item[0] for item in lst] 

In [56]:
def R3_test():
    print("Test-R3: High stakers earn more reward than low stakers")
    num = int(input("enter the number of blocks you want to test: "))
    
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    print("current and last block numbers", block, last_block)
    while block < last_block:
        block = getBlockNumber()
        
    new_epoch_block = block + 2 # first block is problematic, hence going for second.
    while block < new_epoch_block:
        block = getBlockNumber()
    
    print("new epoch first block reached", new_epoch_block, "will begin testing")
    new_epoch_block = block + 1
    
    rewards, stakes, shards = getStakeAndRewards()
    
    print("obtained first block stakes and rewards", "total stakes found=", \
          len(stakes), "total rewards found=", len(rewards))
    
    while block < new_epoch_block:
        block = getBlockNumber()
        
    print("new epoch second block reached", block, "will begin comparing stakes and rewards")  

    iterations = 0
    while iterations < num:
        new_rewards = getRewards()
        last_block = block
        current_block = last_block
        
        key_to_stake = dict()
        key_to_reward = dict()
        key_to_shard = dict()
        for addr, reward in new_rewards.items():
            if addr in rewards and addr in stakes and addr in shards:
                addr_reward = reward - rewards[addr]
                slots = len(stakes[addr])
                per_slot_reward = addr_reward / slots
                for key, stake in stakes[addr].items():
                    key_to_reward[key] = per_slot_reward
                    if key in stakes[addr]:
                        key_to_stake[key] = stakes[addr][key]
                    if key in shards[addr]:
                        key_to_shard[key] = shards[addr][key]
        
        shard_rewards = dict()
        shard_stakes = dict()
        for key, shard in key_to_shard.items():
            if shard not in shard_stakes:
                shard_stakes[shard] = dict()
            if shard not in shard_rewards:
                shard_rewards[shard] = dict()
            shard_stakes[shard][key] = key_to_stake[key]
            shard_rewards[shard][key] = key_to_reward[key]
            
        for shard in shard_rewards.keys():
            sorted_stakes = sorted(shard_stakes[shard].items(), key=lambda kv: kv[1], reverse = True)
            sorted_rewards = sorted(shard_rewards[shard].items(), key=lambda kv: kv[1], reverse = True)
#             print(sorted_stakes)
#             print(sorted_rewards)
            stake_keys = extract(sorted_stakes)
            reward_keys = extract(sorted_rewards)
            print("comparison to begin, two lengths:", len(stake_keys), len(reward_keys))
            if proportional(stake_keys, reward_keys) == False:
                print("Test-R3: on shard", shard, "failed")
                print("validators sorted by stakes: ", stake_keys)
                print("validators sorted by reward: ", reward_keys)
            else:
                print("Test-R3: on shard: ", shard, "succeeded")
            
        last_block = current_block
        current_block = getBlockNumber()
        while current_block == last_block:
            current_block = getBlockNumber()
        
        rewards = new_rewards
        iterations = iterations + 1
        if current_block > last_block:
            break

In [57]:
R3_test()

Test-R3: High stakers earn more reward than low stakers
enter the number of blocks you want to test: 1
current and last block numbers 3780 3799
new epoch first block reached 3801 will begin testing
obtained first block stakes and rewards total stakes found= 39 total rewards found= 39
new epoch second block reached 3802 will begin comparing stakes and rewards
comparison to begin, two lengths: 24 24
Test-R3: on shard 1 failed
validators sorted by stakes:  ['7a4a5b3881a9f64851c9b5ec5c449b293ce567dec39bd3e80b9726cc12825e0eec0ef2bc682114e2f123ad7e1bcc6b99', '85dde30f23ffd30b511582550a20d675e96beaf348af5c1401eaef69e8485ac867cd78e9d4e59f42d9c65b8ae4466981', 'c221b753a0411f6de095610e4dcbf1be1788821973bb5b506ff4956e9b9b60cf331f83ba1dc40ddc17ef25e107bdfb0d', '4ee50030f8f372bc2e605ed20a5c80ad877e65f8d566f0d725c3fc0cc0a9fb67562785731d2585b3603b189a603ffc8d', '146fd89c9cd2f4c3b30cafc3c0fc7aae8cb3ab951b1f7008f1d5eaede2b5699d6979f42196f10e2760b9fc81c1482901', '65548d2a749327506a910a7307cf770770854ef5